In [12]:
##Import libraries
import torch
import pandas as pd
import torch.nn as nn
import torch.nn.functional as F
import torch
import numpy as np
import random
import time
import datetime
import os
import sys
import csv
from torch import backends
from torch.autograd import Variable
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

In [94]:
##SETTINGS
doExtractBaso = True

nfold = 10 # Number of folds 
fold_offset = 1 # Starts from fold 'fold_offset'
lr=0.01 #learning rate

batch_size = 32
val_split = 0.2 #trainset percentage allocated for devset
test_val_split = 0.1 #trainset percentage allocated for test_val set (i.e. the test set of known patients)

#cwd = os.getcwd()
cwd = "../subjects_intra/min-max/windows_20/tr-False/folds_intra/"
subject = 2 # serve per caricare le folds da cartelle diverse
prefix_train = 'TrainFold'
prefix_test = 'TestFold'

spw=20 #samples per window
nmuscles=10 #initial number of muscles acquired



# Modelli del paper: 11 (FF2), 14 (FF4), 16 (FF5) --> Prova questi!
# FF6 per testarlo potente dopo (17)
model_lst = ['FF','FC2','FC2DP','FC3','FC3dp','Conv1d','MultiConv1d',
             'MultiConv1d_2','MultiConv1d_3', 'MultiConv1d_4', 'MultiConv1d_5', 
             'FF2', 'CNN1', 'FF3', 'FF4', 'CNN2', 'FF5', 'FF6', 'CNN3', 'CNN1-FF5', 'CNN1-2','CNN1-1', 'CNN1-3', 'CNN_w60']
model_select = [17] 

#Early stop settings
maxepoch = 100
maxpatience = 10

use_cuda = False
use_gputil = False
cuda_device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [150]:
## -- Load Folds
## -- This script loads in memory the ten folds (UNLEARNED) previously created
## -- To be further predicted by the ANN and then reconstructed

subject_select = 25
nfold = 10
fold_offset = 1
window_length = 20

prefix_test = 'subject' + str(subject_select) + '/TestFold'
prefix_cut = 'subject' + str(subject_select) + '/s'

# This variable will contain the unlearned piece of signal for each fold
testfolds = []

cuts = pd.read_csv(os.path.join(cwd, prefix_cut + str(subject_select) + '_fold_log.csv'), header = None)
cutting_points = cuts.to_numpy()
pace = int(cutting_points[1] - cutting_points[0])

cutting_samples_start = []
cutting_samples_end = []

print("## -- SUBJECT " + str(subject_select) + " -- #\n")
for j in range(fold_offset,fold_offset + nfold):
    testdata = pd.read_csv(os.path.join(cwd, prefix_test + str(j) + '.csv'),sep=',',header=None, dtype=np.float32)
    testfolds.append(testdata)
    print("Fold " + str(j) + ": cutting point at window " + str(cutting_points[j-1]) + " -- Sample # " + str(abs(int(20*pace - 20*cutting_points[j-1])) + 1) + "--" + str(int(20*cutting_points[j-1])) + " in the original signal")
    
    # Salvo i cutting points
    cutting_samples_start.append(abs(int(20*pace - 20*cutting_points[j-1])) + 1)
    cutting_samples_end.append(int(20*cutting_points[j-1]))
n_muscles=int((len(testdata.columns)-1)/window_length) 

## -- Salvo i cutting points su un file
a = list(zip(cutting_samples_start,cutting_samples_end))
aa = pd.DataFrame(a)
aa = aa.to_csv('../subjects_intra/min-max/windows_20/tr-False/folds_intra/subject' + str(subject_select) + '/cutting_points.csv', index = None, header = None)

print("\nThe number of column for each window is " + str(len(testdata.columns)))
print("The number of muscles is " + str(n_muscles))

## -- SUBJECT 25 -- #

Fold 1: cutting point at window [1356] -- Sample # 1--27120 in the original signal
Fold 2: cutting point at window [2712] -- Sample # 27121--54240 in the original signal
Fold 3: cutting point at window [4068] -- Sample # 54241--81360 in the original signal
Fold 4: cutting point at window [5424] -- Sample # 81361--108480 in the original signal
Fold 5: cutting point at window [6780] -- Sample # 108481--135600 in the original signal
Fold 6: cutting point at window [8136] -- Sample # 135601--162720 in the original signal
Fold 7: cutting point at window [9492] -- Sample # 162721--189840 in the original signal
Fold 8: cutting point at window [10848] -- Sample # 189841--216960 in the original signal
Fold 9: cutting point at window [12204] -- Sample # 216961--244080 in the original signal
Fold 10: cutting point at window [13560] -- Sample # 244081--271200 in the original signal


PermissionError: [Errno 13] Permission denied: '../subjects_intra/min-max/windows_20/tr-False/folds_intra/subject25/cutting_points.csv'

In [111]:
## -- Salva il basografico predetto su un file sfruttando l'accuracy

def save_predicted_baso(loader, model, subject_select, model_select, num_fold):
    windows_length = 20
    predicted_baso_windowed = []
    with torch.no_grad():
        
        ## -- Estrae gli output di ciascuna window
        for i, data, in enumerate(loader, 0):
            inputs, labels = data
            outputs = model(inputs)
            outputs[outputs >= 0.5] = 1
            outputs[outputs < 0.5] = 0
            predicted_temp = outputs.tolist()
            predicted_baso_windowed.extend(predicted_temp)
        
        predicted_baso_windowed = pd.DataFrame(predicted_baso_windowed)    # Converto in DataFrame
        predicted_baso_windowed = predicted_baso_windowed.as_matrix()
        
        limit = len(predicted_baso_windowed)
        predicted_baso = np.empty([limit*windows_length,1], dtype = int)
        
        count = 0
        for j in range(0, limit):
            predicted_baso[count:count+windows_length] = predicted_baso_windowed[j]
            count += windows_length
            
        predicted_baso = pd.DataFrame(predicted_baso)
        out_path = '../subjects_intra/min-max/windows_20/tr-False/folds_intra/subject' + str(subject_select) + "/Predicted/"  
        if not os.path.exists(out_path):
            os.makedirs(out_path)
        predicted_baso = predicted_baso.to_csv(out_path + str(num_fold) + '_predicted.csv', index = None, header = None)

In [97]:
import models
from models import *
models._spw = spw
models._nmuscles = nmuscles
models._batch_size = batch_size

#Seeds
def setSeeds(seed):
    torch.manual_seed(seed)
    random.seed(seed)
    np.random.seed(seed)
    
setSeeds(0)

In [110]:
torch.backends.cudnn.benchmark = True

def train_test():
    print('Model ' + str(model_select) + ' - Subject ' + str(subject_select) + '\n')
    for k in model_select:
            for i in range(1,nfold+1):
                
               # t0 = time.time()
                setSeeds(0)
                
                # Classe per il Testset UNLEARNED
                class Testdataset_U(Dataset):
                    def __init__(self):
                        self.data=testfolds[i-1]
                        self.x_data=torch.from_numpy(np.asarray(self.data.iloc[:, 0:-1]))
                        self.len=self.data.shape[0]
                        self.y_data = torch.from_numpy(np.asarray(self.data.iloc[:, [-1]]))
                        if (use_cuda):
                            self.x_data = self.x_data.cuda()
                            self.y_data = self.y_data.cuda()
                    def __getitem__(self, index):
                        return self.x_data[index], self.y_data[index]
                    def __len__(self):
                        return self.len

                ## DEFINISCO I DATASET DA CARICARE    
                testdataset_U = Testdataset_U()    # Test dataset UNLEARNED
                test_loader = torch.utils.data.DataLoader(testdataset_U, batch_size = batch_size, 
                                                          drop_last = True)
                #print('Test Set (U) dimension: ' + str(len(test_loader)))
                
                modelClass = "Model" + str(k)
                model = eval(modelClass)()
                
                if (use_cuda):
                    model = model.cuda()
                
                fold_predicted = i
                model_folder = "../subjects_intra/min-max/windows_20/tr-False/folds_intra/subject" + str(subject_select) + "/Report_FF6/"
                
                if doExtractBaso:
                    if use_cuda:                        
                        state = torch.load(os.path.join(model_folder,'F'+str(i)+'best.pth.tar'))
                    else:
                        state = torch.load(os.path.join(model_folder,'F'+str(i)+'best.pth.tar'), map_location=lambda storage, loc: storage)
                    stop_epoch = state['epoch']
                    model.load_state_dict(state['state_dict'])
                    if not use_cuda:
                        model.cpu()
                    accuracy_dev = state['best_acc_dev']
                    model.eval()
                    
                    ## -- QUESTO DEVI CONTROLLARLO! SE ESCE CHE i È DIVERSO DALLA FOLD DEVI RIFARE TUTTO!
                    
                    print('F' + str(i) + 'best.pth.tar')
                    predicted_baso = save_predicted_baso(test_loader, model, subject_select, model_lst[k], fold_predicted)   
            

        

In [149]:
#nmuscles=int((len(traindata.columns)-1)/spw)
if use_cuda and not use_gputil and cuda_device!=None and torch.cuda.is_available():
    with torch.cuda.device(cuda_device):
        print('I am using CUDA: SUCCESS!')
        train_test()
else:
    print('I am NOT using CUDA: SUCCESS!')
    train_test()

I am NOT using CUDA: SUCCESS!
Model [17] - Subject 24

F1best.pth.tar


C:\Users\Nabuconodosor II\Miniconda3\lib\site-packages\ipykernel_launcher.py:18: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


F2best.pth.tar
F3best.pth.tar
F4best.pth.tar
F5best.pth.tar
F6best.pth.tar
F7best.pth.tar
F8best.pth.tar
F9best.pth.tar
F10best.pth.tar
